In [49]:
"""
Approach:
1 - load all numbers from all reports
2 - line-by-line, digit-by-digit populating of JSON file
3 - get it to work, refactor later as needed.
"""

import os
import json
import pandas as pd
import arcpy
arcpy.env.workspace = r'I:\Projects\Darren\PPA_V2_GIS\PPA_V2.gdb'

import ppa_input_params as params
from accessibility_calcs import get_acc_data
from mix_index_for_project import get_mix_idx
from landuse_buff_calcs import LandUseBuffCalcs

# load json template
in_json = r"C:\Users\dconly\GitRepos\PPA3\testing\json_output\SACOG_ReduceVMT_template.json"
with open(in_json, "r") as j_in:
    json_loaded = json.load(j_in)


# specify project line feature class and attributes
project_fc = r'I:\Projects\Darren\PPA_V2_GIS\PPA_V2.gdb\Polylines'

ptype = params.ptype_arterial
data_years = [2016, 2040]
geo_project = "Project"
geo_ctype = "Community Type"
geo_region = "Region"


# csv table of aggregate values
aggval_csv = r"C:\Users\dconly\GitRepos\PPA2\ppa\Input_Template\CSV\Agg_ppa_vals04222020_1017.csv"

# re-used json keys
k_charts = "charts"
k_features = "features" # remember, this is a list of dicts
k_attrs = "attributes"
k_year = "year"
k_type = "type"

geographies = [geo_project, geo_ctype, geo_region]


In [44]:
# identify project community type
def get_proj_ctype(in_project_fc, commtypes_fc):
    '''Get project community type, based on which community type has most spatial overlap with project'''
    temp_intersect_fc = os.path.join(arcpy.env.scratchGDB, 'temp_intersect_fc') # don't use "memory", use scratch GDB
    arcpy.Intersect_analysis([in_project_fc, commtypes_fc], temp_intersect_fc, "ALL", 
                             0, "LINE")
    
    # debugging messages to find out why ctype tagging intermittently fails
    intersect_cnt = int(arcpy.GetCount_management(temp_intersect_fc)[0])
    in_project_cnt = int(arcpy.GetCount_management(in_project_fc)[0])
    arcpy.AddMessage("project line feature count: {}".format(in_project_cnt))
    arcpy.AddMessage("Project segments after intersecting with comm types: {}".format(intersect_cnt))
    
    len_field = 'SHAPE@LENGTH'
    fields = ['OBJECTID', len_field, params.col_ctype]
    ctype_dist_dict = {}
    
    with arcpy.da.SearchCursor(temp_intersect_fc, fields) as cur:
        for row in cur:
            ctype = row[fields.index(params.col_ctype)]
            seg_len = row[fields.index(len_field)]
        
            if ctype_dist_dict.get(ctype) is None:
                ctype_dist_dict[ctype] = seg_len
            else:
                ctype_dist_dict[ctype] += seg_len
    try:
        maxval = max([v for k, v in ctype_dist_dict.items()])
        proj_ctype = [k for k, v in ctype_dist_dict.items() if v == maxval][0]

        return proj_ctype
    except:
        raise ValueError("ERROR: No Community Type identified for project. \n{} project line features." \
                         " {} features in intersect layer.".format(in_project_cnt, in_project_cnt))
        
project_commtype = get_proj_ctype(project_fc, params.comm_types_fc)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000725: Output Feature Class: Dataset C:\Users\dconly\AppData\Local\Temp\scratch.gdb\temp_intersect_fc already exists.
Failed to execute (Intersect).


In [45]:
# get all applicable comm type and region aggregate values
df_agg = pd.read_csv(aggval_csv)
metname_col = "metric_name"
df_agg = df_agg.rename(columns={'Unnamed: 0':metname_col})

In [46]:
# calc land use buffer values 
buffdist_ft = 2640
value_fields = ['EMPTOT', 'DU_TOT']

out_data = {}
for data_year in data_years:
    in_pcl_pt_fc = params.parcel_pt_fc_yr(data_year)
    year_dict = LandUseBuffCalcs(in_pcl_pt_fc, project_fc, ptype, value_fields, buffdist_ft).point_sum()
    out_data[data_year] = year_dict
    

# update applicable field values in JSON template
for i, year in enumerate(data_years):
    jobs = out_data[year]['EMPTOT']
    du = out_data[year]['DU_TOT']
    json_loaded[k_charts]["Jobs and Dwelling"][k_features][i][k_attrs]['year'] = year
    json_loaded[k_charts]["Jobs and Dwelling"][k_features][i][k_attrs]['jobs'] = jobs
    json_loaded[k_charts]["Jobs and Dwelling"][k_features][i][k_attrs]['dwellingUnits'] = du

# print(json.dumps(json_loaded, indent=4))

In [50]:
# calc accessibility

fc_accessibility_data = params.accdata_fc

# project level dict
dict_data = get_acc_data(project_fc, fc_accessibility_data, ptype)
acc_metrics = {'WALKDESTSpoi2':'30 Min Walk', 'BIKEDESTSpoi2':'30 Min Biking', 
                'AUTODESTSpoi2':'15 Min Drive', 'TRANDESTSpoi2':'45 Min Transit'}
dict_data2 = {k:dict_data[k] for k in acc_metrics.keys()}
# print(dict_data2)

for i, k in enumerate(list(acc_metrics.keys())):
    json_loaded[k_charts]["Base Year Service Accessibility"][k_features][i][k_attrs][k_type] = acc_metrics[k]
    json_loaded[k_charts]["Base Year Service Accessibility"][k_features][i][k_attrs][geo_project] = dict_data2[k]

print(json_loaded)
# aggdatadf_proj = df_agg.loc[df_agg[metname_col].isin(vmt_metrics)][[metname_col, project_commtype, 'year']]




{'charts': {'Jobs and Dwelling': {'title': 'Total Jobs and Dwelling Units within 0.5mi of project', 'features': [{'attributes': {'year': '2016', 'jobs': 1612, 'dwellingUnits': 3147}}, {'attributes': {'year': '2040', 'jobs': 2303, 'dwellingUnits': 4516}}]}, 'Land Use Diversity': {'title': 'Land Use Diversity within 1mi of Project', 'features': [{'attributes': {'type': 'Project', 'diversity 2016': 0.51, 'diversity 2040': 0.56}}, {'attributes': {'type': 'Community Type', 'diversity 2016': 0.74, 'diversity 2040': 0.74}}, {'attributes': {'type': 'Region', 'diversity 2016': 1.0, 'diversity 2040': 0.93}}]}, 'Base Year Service Accessibility': {'title': 'Land Use Diversity within 1mi of Project', 'features': [{'attributes': {'type': '30 Min Walk', 'Project': 22.529602076124565, 'community Type': 17, 'region': 17}}, {'attributes': {'type': '30 Min Biking', 'Project': 190.794785467128, 'community Type': 147, 'region': 140}}, {'attributes': {'type': '15 Min Drive', 'Project': 943.9961937716263, 'c

In [37]:
aggdatadf_proj = df_agg.loc[df_agg[metname_col].isin(vmt_metrics)][[metname_col, project_commtype, 'year']]
aggdatadf_proj.to_dict(orient='dict')

{'metric_name': {8: 'WALKDESTSpoi2',
  9: 'BIKEDESTSpoi2',
  10: 'AUTODESTSpoi2',
  11: 'TRANDESTSpoi2'},
 'Arterials & Suburban Corridors': {8: 23.408610137958803,
  9: 199.25458633185085,
  10: 686.1842636258035,
  11: 122.29326086755307},
 'year': {8: 2016, 9: 2016, 10: 2016, 11: 2016}}

In [10]:
# calc mix index
buff_dist_ft = params.mix_index_buffdist  # distance in feet--MIGHT NEED TO BE ADJUSTED FOR WGS 84--SEE OLD TOOL FOR HOW THIS WAS RESOLVED

out_dict = {}
for data_year in data_years:
    in_pcl_pt_fc = params.parcel_pt_fc_yr(in_year=data_year) # input fc of parcel data--must be points!
    year_dict = get_mix_idx(in_pcl_pt_fc, project_fc, ptype)
    out_dict[data_year] = year_dict

print(out_dict)

{2016: {'mix_index': 0.49111228436332865}, 2040: {'mix_index': 0.5444755587594678}}
